In [1]:
import veloxchem as vlx

In [2]:
class MockTask:
    def __init__(self, mol, basis, comm, ostream):

        self.molecule = mol
        self.ao_basis = basis
        self.mpi_comm = comm
        self.ostream = ostream

In [3]:
import adcc
from mpi4py import MPI
import sys
import numpy
numpy.set_printoptions(precision=7, suppress=True)

In /home/emi/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/emi/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/emi/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In /home/emi/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/emi/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classi

In [4]:
#Input settings
molecule_string = """
    O 0 0 0
    H 0 0 1.795239827225189
    H 1.693194615993441 0 -0.599043184453037"""

# molecule_string = """
#    Li 0 0 0
#    H 0 0 1.795239827225189
#    """

basis_set_label = 'STO-3G'
scf_settings = {'conv_thresh': 1.0e-8}
method_settings = {'xcfun': 'blyp', 'grid_level': 4}
rsp_settings = {'conv_thresh': 1.0e-6, 'nstates': 3}

In [5]:
# Communicator and output stream
comm = MPI.COMM_WORLD
ostream = vlx.OutputStream(sys.stdout)

In [6]:
# Molecule and basis set
molecule = vlx.Molecule.read_str(molecule_string, units='au')
basis = vlx.MolecularBasis.read(molecule, basis_set_label)

ostream.print_block(molecule.get_string())
ostream.print_block(basis.get_string('Atomic Basis', molecule))
ostream.flush()

                                              Molecular Geometry (Angstroms)                                              
                                                                                                                          
                          Atom         Coordinate X          Coordinate Y          Coordinate Z                           
                                                                                                                          
                           O           0.000000000000        0.000000000000        0.000000000000                         
                           H           0.000000000000        0.000000000000        0.950000004704                         
                           H           0.896000004436        0.000000000000       -0.317000001569                         
                                                                                                                          
                

In [7]:
# SCF
task = MockTask(molecule, basis, comm, ostream)
scfdrv = vlx.ScfRestrictedDriver(comm, ostream)
scfdrv.update_settings(scf_settings, method_settings)
scfdrv.compute(molecule, basis)
scfdrv.task = task

                                                                                                                          
                                            Self Consistent Field Driver Setup                                            
                                                                                                                          
                   Wave Function Model             : Spin-Restricted Kohn-Sham                                            
                   Initial Guess Model             : Superposition of Atomic Densities                                    
                   Convergence Accelerator         : Two Level Direct Inversion of Iterative Subspace                     
                   Max. Number of Iterations       : 50                                                                   
                   Max. Number of Error Vectors    : 10                                                                   
                

In [8]:
# SCF first-order properties
scf_prop = vlx.ScfFirstOrderProperties(comm, ostream)
scf_prop.compute(molecule, basis, scfdrv.scf_tensors)
scf_prop.print_properties(molecule)

                                                                                                                          
                                                Ground-State Dipole Moment                                                
                                               ----------------------------                                               
                                                                                                                          
                                   X   :         0.522120 a.u.         1.327096 Debye                                     
                                   Y   :        -0.000000 a.u.        -0.000000 Debye                                     
                                   Z   :         0.369243 a.u.         0.938522 Debye                                     
                                 Total :         0.639492 a.u.         1.625425 Debye                                     
                

In [9]:
#0.546315*vlx.veloxchemlib.dipole_in_debye()
0.546315*2

1.09263

In [10]:
# Linear Response
#lr_eig_drv = vlx.LinearResponseEigenSolver(comm, ostream)
#lr_eig_drv.update_settings(rsp_settings, method_settings)
#lr_results = lr_eig_drv.compute(molecule, basis, scfdrv.scf_tensors)
#lr_eig_drv.ostream.flush()

In [11]:
# TDHF/TDA, i.e. CIS
tda_drv = vlx.TDAExciDriver(comm, ostream)
tda_drv.update_settings(rsp_settings, method_settings)
tda_results = tda_drv.compute(molecule, basis, scfdrv.scf_tensors)
tda_drv.ostream.flush()

                                                                                                                          
                                                     TDA Driver Setup                                                     
                                                                                                                          
                               Number of States                : 3                                                        
                               Max. Number of Iterations       : 150                                                      
                               Convergence Threshold           : 1.0e-06                                                  
                               ERI Screening Scheme            : Cauchy Schwarz + Density                                 
                               ERI Screening Threshold         : 1.0e-15                                                  
                

In [12]:
# Get MO coefficients, eigenvalues and eigenvectors

nocc = molecule.number_of_alpha_electrons()
mo = scfdrv.scf_tensors['C'] # MO coefficients
mo_occ = mo[:, :nocc]        # occupied
mo_vir = mo[:, nocc:]        # virtual

nocc = mo_occ.shape[1]
nvir = mo_vir.shape[1]

print(scfdrv.density.alpha_to_numpy(0))
print()
print(scfdrv.scf_tensors['D'][0])
#eig_vals = lr_results["eigenvalues"]
#eig_vecs = lr_results["eigenvectors"]

#eig_vec = eig_vecs[:, 0].copy() # first eigenvector
#print(eig_vec)
print("mo_occ:")
print(mo_occ)
print()
print("mo_vir:")
print(mo_vir)

[[ 1.053001  -0.2233136 -0.0144665 -0.0144514 -0.         0.031617
   0.0447147]
 [-0.2233136  0.9951794 -0.0238208 -0.0237819  0.        -0.1791314
  -0.253282 ]
 [-0.0144665 -0.0238208  0.2994026 -0.0914558 -0.         0.3559113
   0.0442778]
 [-0.0144514 -0.0237819 -0.0914558  0.2995045  0.        -0.0769756
   0.3503361]
 [-0.         0.        -0.         0.         1.         0.
  -0.       ]
 [ 0.031617  -0.1791314  0.3559113 -0.0769756  0.         0.4489835
   0.1265415]
 [ 0.0447147 -0.253282   0.0442778  0.3503361 -0.         0.1265415
   0.5384422]]

[[ 1.053001  -0.2233136 -0.0144665 -0.0144514 -0.         0.031617
   0.0447147]
 [-0.2233136  0.9951794 -0.0238208 -0.0237819  0.        -0.1791314
  -0.253282 ]
 [-0.0144665 -0.0238208  0.2994026 -0.0914558 -0.         0.3559113
   0.0442778]
 [-0.0144514 -0.0237819 -0.0914558  0.2995045  0.        -0.0769756
   0.3503361]
 [-0.         0.        -0.         0.         1.         0.
  -0.       ]
 [ 0.031617  -0.1791314  0.355

In [13]:
print(scfdrv.scf_tensors.keys())
# Transform the Fock matrix from AO to MO basis
fock = scfdrv.scf_tensors['F'][0]
ovlp = scfdrv.scf_tensors['S']
print(numpy.matmul(mo.T, numpy.matmul(ovlp, mo)))
print(numpy.matmul(mo.T, numpy.matmul(fock, mo)))

dict_keys(['C', 'E', 'S', 'D', 'F'])
[[ 1. -0. -0. -0. -0.  0. -0.]
 [-0.  1.  0.  0. -0. -0.  0.]
 [-0.  0.  1.  0. -0.  0.  0.]
 [-0.  0.  0.  1.  0. -0. -0.]
 [-0. -0. -0.  0.  1. -0.  0.]
 [ 0. -0.  0. -0. -0.  1.  0.]
 [-0.  0.  0. -0.  0.  0.  1.]]
[[-18.4756378  -0.          0.          0.         -0.          0.
   -0.       ]
 [ -0.         -0.8343382  -0.         -0.         -0.         -0.
   -0.       ]
 [  0.         -0.         -0.3942781  -0.         -0.          0.
   -0.       ]
 [  0.         -0.         -0.         -0.1430215  -0.         -0.
   -0.       ]
 [ -0.         -0.         -0.         -0.         -0.0627808  -0.
    0.       ]
 [  0.         -0.          0.         -0.         -0.          0.3077275
    0.       ]
 [ -0.         -0.         -0.         -0.          0.          0.
    0.4394891]]


In [14]:
# Convert the first eigenvector to matrix form
#eig_vec_as_mat = vlx.LinearResponseEigenSolver.lrvec2mat(eig_vec, nocc, nocc + nvir)

In [15]:
tda_eig_vals = tda_results["eigenvalues"]
tda_eig_vecs = tda_results["eigenvectors"]
tda_size = tda_eig_vecs[:,0].shape
tda_eig_vec=tda_eig_vecs[:,0].copy()
tda_eig_vec_as_mat = tda_eig_vec.reshape(nocc, nvir) #/ numpy.sqrt(2.0)

In [16]:
print(tda_eig_vec)

[-0.        -0.         0.         0.        -0.         0.
 -0.         0.         0.9999999  0.0004396]


In [17]:
#tda_eig_vec_as_mat = vlx.LinearResponseEigenSolver.lrvec2mat(tda_eig_vec, nocc, nocc + nvir)

In [18]:
print(tda_eig_vec_as_mat)

[[-0.        -0.       ]
 [ 0.         0.       ]
 [-0.         0.       ]
 [-0.         0.       ]
 [ 0.9999999  0.0004396]]


In [19]:
adc1 = adcc.run_adc(scfdrv, method='adc1', n_singlets=3, conv_tol=1e-4)

Starting adc1 singlet Jacobi-Davidson ...
Niter n_ss  max_residual  time  Ritz values
  1     6     0.0028151   54ms  [-0.1401709 -0.0772794  0.006549 ]
  2    10    9.5173e-30   48ms  [-0.1401709 -0.0772794  0.0031472]
=== Converged ===
    Number of matrix applies:    10
    Total solver time:           105.845ms


In [20]:
adc1_eig_vec = adc1.excitation_vector[0]["ph"].to_ndarray()
adc1_eig_vals = adc1.excitation_energy
adc1.state_dipole_moment

array([[-0.2243137, -0.       , -0.1502563],
       [ 0.0191267,  0.       ,  0.0056733],
       [-0.4335008,  0.       , -0.300666 ]])

In [21]:
print(adc1_eig_vec)
print(adc1_eig_vals)
print(adc1.describe())

[[ 0.        -0.         0.         0.       ]
 [ 0.         0.         0.         0.       ]
 [ 0.        -0.         0.         0.       ]
 [ 0.         0.         0.         0.       ]
 [-0.707106  -0.0010372  0.         0.       ]
 [ 0.         0.         0.        -0.       ]
 [ 0.         0.         0.         0.       ]
 [ 0.         0.         0.        -0.       ]
 [ 0.         0.         0.         0.       ]
 [ 0.         0.        -0.707106  -0.0010372]]
[-0.1401709 -0.0772794  0.0031472]
+----------------------------------------------------+
| adc1                          singlet ,  converged |
+----------------------------------------------------+
|  #        excitation energy     osc str    |v1|^2  |
|          (au)           (eV)                       |
|  0    -0.1401709     -3.814245   0.0008         1  |
|  1   -0.07727935     -2.102878   0.0000         1  |
|  2   0.003147182     0.0856392   0.0003         1  |
+----------------------------------------------------+

In [22]:
print(tda_eig_vecs[:,0])
print(tda_eig_vals)

[-0.        -0.         0.         0.        -0.         0.
 -0.         0.         0.9999999  0.0004396]
[0.413264  0.4984099 0.5178776]


In [23]:
numpy.linalg.norm(adc1_eig_vec)

0.9999999999999998

In [24]:
0.9999995/numpy.sqrt(2.0)

0.7071064276331569

In [25]:
0.0009539/numpy.sqrt(2.0)

0.0006745091585738476

In [26]:
print(adc1_eig_vec.shape)
print(adc1_eig_vec)
print()
print(tda_eig_vec_as_mat.shape)
print(tda_eig_vec_as_mat/numpy.sqrt(2))

(10, 4)
[[ 0.        -0.         0.         0.       ]
 [ 0.         0.         0.         0.       ]
 [ 0.        -0.         0.         0.       ]
 [ 0.         0.         0.         0.       ]
 [-0.707106  -0.0010372  0.         0.       ]
 [ 0.         0.         0.        -0.       ]
 [ 0.         0.         0.         0.       ]
 [ 0.         0.         0.        -0.       ]
 [ 0.         0.         0.         0.       ]
 [ 0.         0.        -0.707106  -0.0010372]]

(5, 2)
[[-0.        -0.       ]
 [ 0.         0.       ]
 [-0.         0.       ]
 [-0.         0.       ]
 [ 0.7071067  0.0003108]]


In [27]:
vlx_dm_oo = -numpy.einsum('ia,ja->ij', tda_eig_vec_as_mat, tda_eig_vec_as_mat)
vlx_dm_vv = numpy.einsum('ia,ib->ab', tda_eig_vec_as_mat, tda_eig_vec_as_mat)
vlx_DM_ovov = -numpy.einsum('ib,ja->iajb', tda_eig_vec_as_mat, tda_eig_vec_as_mat)

print(vlx_dm_oo)

[[-0.  0.  0. -0.  0.]
 [ 0. -0. -0. -0. -0.]
 [ 0. -0. -0. -0.  0.]
 [-0. -0. -0. -0.  0.]
 [ 0. -0.  0.  0. -1.]]


In [28]:
adcc_dm_oo = -numpy.einsum('ia,ja->ij', adc1_eig_vec, adc1_eig_vec)
adcc_dm_vv = numpy.einsum('ia,ib->ab', adc1_eig_vec, adc1_eig_vec)
adcc_DM_ovov = -numpy.einsum('ib,ja->iajb', adc1_eig_vec, adc1_eig_vec)

#print(adcc_dm_oo)

foo = adc1.reference_state.fock("o1o1").to_ndarray()
fvv = adc1.reference_state.fock("v1v1").to_ndarray()
ovov = adc1.reference_state.eri("o1v1o1v1").to_ndarray()

adcc_excitation_energy = (numpy.einsum('ij,ij->',adcc_dm_oo,foo)
                     + numpy.einsum('ab,ab->', adcc_dm_vv, fvv)
                     + numpy.einsum('iajb,iajb->', adcc_DM_ovov, ovov)
                    )

In [29]:
print(adcc_excitation_energy)

-0.14017092686167798


$E = \frac{1}{4} \sum_{pqrs} \Gamma_{pqrs} \langle pq || rs \rangle$

$\langle ia || jb \rangle = - \langle ai || jb \rangle = - \langle ia || bj \rangle = \langle ai || bj \rangle$

analogous symmetry for $\boldsymbol{\Gamma}$

In [30]:
vlx_excitation_energy = (1.0*numpy.einsum('ij,ij->', vlx_dm_oo,foo[:nocc,:nocc])
                        +1.0*numpy.einsum('ab,ab->', vlx_dm_vv, fvv[:nvir,:nvir])
                        +1.0*(
                        +1.0*numpy.einsum('iajb,iajb->', vlx_DM_ovov, ovov[:nocc, :nvir, :nocc, :nvir])##aaaa
                        #+0.0*numpy.einsum('iajb,iajb->', vlx_DM_ovov, ovov[:nocc, :nvir, :nocc, nvir:])# zero block
                        #+1.0*numpy.einsum('iajb,iajb->', vlx_DM_ovov, ovov[:nocc, :nvir, nocc:, :nvir])# zero block
                        #+1.0*numpy.einsum('iajb,iajb->', vlx_DM_ovov, ovov[:nocc, :nvir, nocc:, nvir:])# zero block
                        #+1.0*numpy.einsum('iajb,iajb->', vlx_DM_ovov, ovov[:nocc, nvir:, :nocc, :nvir])# zero block
                        #+1.0*numpy.einsum('iajb,iajb->', vlx_DM_ovov, ovov[:nocc, nvir:, :nocc, nvir:])# zero block
                        +1.0*numpy.einsum('iajb,iajb->', vlx_DM_ovov, ovov[:nocc, nvir:, nocc:, :nvir])##bbbb
                        #+1.0*numpy.einsum('iajb,iajb->', vlx_DM_ovov, ovov[:nocc, nvir:, nocc:, nvir:])# zero block
                        #+1.0*numpy.einsum('iajb,iajb->', vlx_DM_ovov, ovov[nocc:, :nvir, :nocc, :nvir])# zero block
                        #+1.0*numpy.einsum('iajb,iajb->', vlx_DM_ovov, ovov[nocc:, :nvir, :nocc, nvir:])##abba
                        #+1.0*numpy.einsum('iajb,iajb->', vlx_DM_ovov, ovov[nocc:, :nvir, nocc:, :nvir])# zero block
                        #+1.0*numpy.einsum('iajb,iajb->', vlx_DM_ovov, ovov[nocc:, nvir:, :nocc, :nvir])# zero block
                        #+1.0*numpy.einsum('iajb,iajb->', vlx_DM_ovov, ovov[nocc:, nvir:, :nocc, nvir:])# zero block
                        #+1.0*numpy.einsum('iajb,iajb->', vlx_DM_ovov, ovov[nocc:, nvir:, nocc:, :nvir])# zero block
                        #+1.0*numpy.einsum('iajb,iajb->', vlx_DM_ovov, ovov[nocc:, nvir:, nocc:, nvir:])##baab
                        ))
print("vlx energy =", vlx_excitation_energy)

vlx energy = -0.14017086048691507


In [31]:
print(vlx_excitation_energy)

-0.14017086048691507


In [32]:
adcc_excitation_energy = (1.0*numpy.einsum('ij,ij->', vlx_dm_oo,foo[:nocc,:nocc])
                        +1.0*numpy.einsum('ab,ab->', vlx_dm_vv, fvv[:nvir,:nvir])
                        +2.0*numpy.einsum('iajb,iajb->', adcc_DM_ovov[:nocc, :nvir, :nocc, :nvir], ovov[:nocc, :nvir, :nocc, :nvir])
                        +2.0*numpy.einsum('iajb,iajb->', adcc_DM_ovov[:nocc, :nvir, :nocc, :nvir], ovov[:nocc, nvir:, nocc:, :nvir])
    )
print(adcc_excitation_energy)

-0.14017118491998384


In [33]:
print(ovov.shape)
#print(ovov)

(10, 4, 10, 4)


In [34]:
#print(vlx_DM_ovov)
#print()
print(vlx_DM_ovov.shape)
#print(adcc_DM_ovov)

(5, 2, 5, 2)


In [35]:
# Making the Vlx 2PDM twice the size with the other spin blocks
# and assign the corresponding values to it by comparing to the adcc matrix
# afterwards divide by two to yield the same numbers
##vlx_DM_ovov_full = numpy.zeros((2*nocc, 2*nvir, 2*nocc, 2*nvir))
#vlx_DM_ovov_full[:nocc, :nvir, :nocc, :nvir] = vlx_DM_ovov
##vlx_DM_ovov_full[0, 0, 0, 0] = vlx_DM_ovov_full[0, 3, 1, 0] = vlx_DM_ovov_full[1,0,0,3] = vlx_DM_ovov_full[1,3,1,3] = vlx_DM_ovov[0,0,0,0]
##vlx_DM_ovov_full[0, 0, 0, 2] = vlx_DM_ovov_full[0, 2, 0, 0] = vlx_DM_ovov_full[0,3,1,2] = vlx_DM_ovov_full[0,5,1,0] = vlx_DM_ovov[0,2,0,0]
##vlx_DM_ovov_full[1, 0, 0, 5] = vlx_DM_ovov_full[1, 2, 0, 3] = vlx_DM_ovov_full[1,3,1,5] = vlx_DM_ovov_full[1,5,1,3] = vlx_DM_ovov[0,2,0,0]
##vlx_DM_ovov_full[0, 2, 0, 2] = vlx_DM_ovov_full[0, 5, 1, 2] = vlx_DM_ovov_full[1,2,0,5] = vlx_DM_ovov_full[1,5,1,5] = vlx_DM_ovov[0,2,0,2]

### matrix elements were twice as large
##vlx_DM_ovov_full *= 0.5
# norm of the difference between the two density matrices does not seem to vanish entirely...
##print("norm of the difference of the 2PDMs = ", numpy.linalg.norm(adcc_DM_ovov - vlx_DM_ovov_full))
#print(vlx_DM_ovov_full)
##vlx_excitation_energy = (1.0*numpy.einsum('ij,ij->', vlx_dm_oo,foo[:nocc,:nocc])
##                        #(1.0*numpy.einsum('ij,ij->', adcc_dm_oo,foo)
##                        #+1.0*numpy.einsum('ij,ij->', adcc_dm_vv,fvv)
##                        +1.0*numpy.einsum('ab,ab->', vlx_dm_vv, fvv[:nvir,:nvir])
##                        +1.0*numpy.einsum('iajb,iajb->', vlx_DM_ovov_full, ovov)
##                        )
##print("vlx exc. energy =", vlx_excitation_energy)
#print("adcc - vlx =", adcc_excitation_energy - vlx_excitation_energy)

In [36]:
print(adcc_DM_ovov.shape)
print()
#print(adcc_DM_ovov)
print()
#print(adcc_DM_ovov - vlx_DM_ovov_full) # here everything appears to be zero

(10, 4, 10, 4)




In [37]:
# Transform the one-particle density matrices from MO to AO basis
vlx_dm_oo_in_ao = numpy.matmul(mo_occ, numpy.matmul(vlx_dm_oo, mo_occ.T))
adcc_dm_oo_in_ao = 2.0*numpy.matmul(mo_occ, numpy.matmul(adcc_dm_oo[:nocc,:nocc], mo_occ.T))

vlx_dm_vv_in_ao = numpy.matmul(mo_vir, numpy.matmul(vlx_dm_vv, mo_vir.T))
adcc_dm_vv_in_ao = 2.0*numpy.matmul(mo_vir, numpy.matmul(adcc_dm_vv[:nvir,:nvir], mo_vir.T))

vlx_dm_ao = vlx_dm_oo_in_ao + vlx_dm_vv_in_ao
adcc_dm_ao = adcc_dm_oo_in_ao + adcc_dm_vv_in_ao

from veloxchem.veloxchemlib import AODensityMatrix
from veloxchem.veloxchemlib import denmat

ao_density_object = AODensityMatrix([vlx_dm_ao], denmat.rest)

print(ao_density_object.beta_to_numpy(0))
print()
print("VLX")
print(vlx_dm_ao)
print("\nADCC")
print(adcc_dm_ao)

contract_fock_mo = (1.0*numpy.einsum('ij,ij->', vlx_dm_oo, foo[:nocc,:nocc])
                    +1.0*numpy.einsum('ab,ab->', vlx_dm_vv, fvv[:nvir,:nvir]))

contract_fock_ao = numpy.einsum('mn,mn->', vlx_dm_ao, fock)

print("contract_fock_ao =", contract_fock_ao)
print("contract_fock_mo =", contract_fock_mo)

[[ 0.0179086 -0.1194165  0.1073324  0.107737   0.        -0.0559578
  -0.079603 ]
 [-0.1194165  0.7962818 -0.7157039 -0.7184018 -0.         0.3731325
   0.5308013]
 [ 0.1073324 -0.7157039  0.6432798  0.6457047  0.        -0.3353742
  -0.477088 ]
 [ 0.107737  -0.7184018  0.6457047  0.6481388 -0.        -0.3366385
  -0.4788865]
 [ 0.        -0.         0.        -0.        -1.         0.
   0.       ]
 [-0.0559578  0.3731325 -0.3353742 -0.3366385  0.         0.1748475
   0.2487301]
 [-0.079603   0.5308013 -0.477088  -0.4788865  0.         0.2487301
   0.353832 ]]

VLX
[[ 0.0179086 -0.1194165  0.1073324  0.107737   0.        -0.0559578
  -0.079603 ]
 [-0.1194165  0.7962818 -0.7157039 -0.7184018 -0.         0.3731325
   0.5308013]
 [ 0.1073324 -0.7157039  0.6432798  0.6457047  0.        -0.3353742
  -0.477088 ]
 [ 0.107737  -0.7184018  0.6457047  0.6481388 -0.        -0.3366385
  -0.4788865]
 [ 0.        -0.         0.        -0.        -1.         0.
   0.       ]
 [-0.0559578  0.3731325 

In [38]:
# Calculate the total density (including HF part)
# and calculated the unrelaxed excited-state dipole moment

# dipole integrals
dipole_drv = vlx.ElectricDipoleIntegralsDriver(comm)
dipole_mats = dipole_drv.compute(molecule, basis)
dipole_ints = (dipole_mats.x_to_numpy(),
              dipole_mats.y_to_numpy(),
              dipole_mats.z_to_numpy())

# electronic contribution
total_density = 2*scfdrv.scf_tensors['D'][0] + vlx_dm_ao
adcc_total_density = 2*scfdrv.scf_tensors['D'][0] + adcc_dm_ao
electronic_dipole = -1.0 * numpy.array(
    [numpy.sum(dipole_ints[d] * total_density) for d in range(3)])
adcc_electronic_dipole = -1.0 * numpy.array(
    [numpy.sum(dipole_ints[d] * adcc_total_density) for d in range(3)])

# nuclear contribution
coords = molecule.get_coordinates()
nuclear_charges = molecule.elem_ids_to_numpy()
nuclear_dipole = numpy.sum(coords.T * nuclear_charges, axis=1)

dipole_moment = (nuclear_dipole + electronic_dipole)
adcc_dipole_moment = (nuclear_dipole + adcc_electronic_dipole)

print("dipole moment [au] =", dipole_moment)
print()
print("adcc dm dipole moment [au] =", adcc_dipole_moment)
print()
print("ADC(1)/6-31G dipole moment for water in Q-Chem:")
print("Dip. moment [a.u.]:  [0.221200,  -0.00000,   0.151505]")
print()
print("ADC(1) dipole moment from adcc:")
print(adc1.state_dipole_moment)

dipole moment [au] = [-0.2230243  0.        -0.1520825]

adcc dm dipole moment [au] = [-0.2243137 -0.        -0.1502563]

ADC(1)/6-31G dipole moment for water in Q-Chem:
Dip. moment [a.u.]:  [0.221200,  -0.00000,   0.151505]

ADC(1) dipole moment from adcc:
[[-0.2243137 -0.        -0.1502563]
 [ 0.0191267  0.         0.0056733]
 [-0.4335008  0.        -0.300666 ]]


In [39]:
# Transform the excitation vectors from MO to AO basis
tda_eig_vec_ao = numpy.matmul(mo_occ, numpy.matmul(tda_eig_vec_as_mat, mo_vir.T))
print(tda_eig_vec_ao)

[[-0.         0.        -0.        -0.         0.         0.
   0.       ]
 [ 0.        -0.         0.         0.        -0.        -0.
  -0.       ]
 [-0.         0.        -0.        -0.         0.         0.
   0.       ]
 [-0.         0.        -0.        -0.         0.         0.
   0.       ]
 [ 0.133823  -0.8923462  0.8020473  0.8050707 -0.        -0.4181477
  -0.5948378]
 [-0.         0.        -0.        -0.         0.        -0.
   0.       ]
 [-0.         0.        -0.        -0.         0.         0.
   0.       ]]


In [40]:
#dir(scfdrv)

In [41]:
# Get the ERI tensor in AO basis (chemists' notation)

nao = mo.shape[0]

pqrs = numpy.zeros((nao, nao, nao, nao))
eri_drv = vlx.ElectronRepulsionIntegralsDriver(comm)
eri_drv.compute_in_mem(molecule, basis, pqrs)
#print("(pq|rs) =")
#print(pqrs)


\begin{align}
\begin{split}
    - \sum_{\sigma \sigma'} \sum_{ijab} x_{i_{\sigma} b_{\sigma}} x_{j_{\sigma'} a_{\sigma'}}
        \langle i_{\sigma} a_{\sigma'} || j_{\sigma'} b_{\sigma} \rangle
    &= - \sum_{ijab} x_{ib} x_{ja} [ 2 (ij|ab) - 4 (ib|aj) ] \\
    &= + \sum_{ijab} x_{ib} x_{ja} [ 4 (ib|aj) - 2 (ij|ab)  ] \\
    &= \sum_{ijab} x_{ib} x_{ja} \sum_{\mu \nu \theta \varphi}
        C_{\mu i} C_{\nu b} C_{\theta a} C_{\varphi j}  [ 4 (\mu \nu | \theta \varphi)
        - 2 (\mu \varphi | \theta \nu) ] \\
    &= \sum_{\mu \nu \theta \varphi} [ 4 (\mu \nu | \theta \varphi)
        - 2 (\mu \varphi | \theta \nu) ]
        \sum_{ib} C_{\mu i} x_{ib} C_{\nu b} \sum_{ja} C_{\varphi j} x_{ja} C_{\theta a} \\
    &= \sum_{\mu \nu \theta \varphi} x_{\mu \nu} x_{\varphi \theta} [ 4 (\mu \nu | \theta \varphi)
        - 2 (\mu \varphi | \theta \nu) ]       \\
    &= \sum_{\mu \nu} x_{\mu \nu} \sum_{\theta \varphi} x_{\varphi \theta} [ 4 (\mu \nu | \theta \varphi)
        - 2 (\mu \varphi | \theta \nu) ]
\end{split}
\end{align}

In [42]:
# Contraction of electron repulsion integrals in AO basis
# First reference value in MO
eri_contract_mo_adcc = numpy.einsum('iajb,iajb->', adcc_DM_ovov, ovov)
print("Contraction in MO adcc:", eri_contract_mo_adcc)

eri_contract_mo_vlx = (+1.0*numpy.einsum('iajb,iajb->', vlx_DM_ovov, ovov[:nocc, :nvir, :nocc, :nvir])##aaaa
                        +1.0*numpy.einsum('iajb,iajb->', vlx_DM_ovov, ovov[:nocc, nvir:, nocc:, :nvir]))##bbbb
print("Contraction in MO vlx: ", eri_contract_mo_vlx)

# Now contract everything in AO
eri_contract_ao = (0.5*numpy.einsum('mn,mn->', tda_eig_vec_ao, numpy.einsum('pt,mntp->mn', tda_eig_vec_ao, 4*pqrs)
                                   - numpy.einsum('pt,mptn->mn', tda_eig_vec_ao, 2*pqrs))
                    )
print("Contraction in AO:     ", eri_contract_ao)

Contraction in MO adcc: -0.5106795576160396
Contraction in MO vlx:  -0.5106792331829708
Contraction in AO:      -0.510679233182971


In [43]:
mo.shape
print(nocc + nvir)

7


## Orbital response, right hand side

### 1. Contributions from the 1PDMs
\begin{equation}
F^\gamma_{ia}=\sum_{\theta,\varphi}C^\alpha_{\varphi i}C^\alpha_{\theta a}\sum_{\mu,\nu,\sigma}\gamma^\sigma_{\mu\nu}(\mu\nu|\theta \varphi)-\sum_{\theta,\varphi}C^\alpha_{\varphi i}C^\alpha_{\theta a}\sum_{\mu,\nu}\gamma^{\alpha}_{\mu\nu}(\mu\varphi|\theta\nu)\\
=\sum_{\theta,\varphi,\sigma}C^\alpha_{\varphi i}C^\alpha_{\theta a}F^{1,\sigma}_{\varphi\theta}-\sum_{\theta,\varphi}C^\alpha_{\varphi i}C^\alpha_{\theta a}F^{2,\alpha}_{\varphi\theta}\\
=\sum_{\theta,\varphi}C^\alpha_{\varphi i}C^\alpha_{\theta a}F_{\varphi\theta}
\end{equation}

In [44]:
F1_vlx = numpy.einsum('mn,mntp->pt', vlx_dm_ao, pqrs)
F2_vlx = numpy.einsum('mn,mptn->pt', vlx_dm_ao, pqrs)
F_1pdm_vlx = F1_vlx - 0.5*F2_vlx
vlx_1pdm_rhs = numpy.einsum('pi,pa->ia', mo_occ, numpy.einsum('ta,pt->pa', mo_vir, F_1pdm_vlx))
print(vlx_1pdm_rhs)

[[-0.0187704 -0.0000132]
 [-0.0886901  0.0000606]
 [ 0.0004499  0.1057719]
 [-0.0857271 -0.0000833]
 [-0.         0.       ]]


In [45]:
#Use AODensityMatrix object to compute a Fock-like matrix
#expect something like F_{\varphi\theta}, equation above
#see linearsolver.py; we have to create a linear solver object:
from veloxchem.linearsolver import LinearSolver
from veloxchem.veloxchemlib import AOFockMatrix
linear_solver_object = LinearSolver(comm, ostream)
eri_dict = linear_solver_object.init_eri(molecule, basis)
# DFT information
dft_dict = linear_solver_object.init_dft(molecule, scfdrv.scf_tensors)
# PE information
pe_dict = linear_solver_object.init_pe(molecule, basis)
timing_dict = {}
fock_matrix_rhs_1pdm = AOFockMatrix(ao_density_object)
print(fock_matrix_rhs_1pdm)
linear_solver_object.comp_lr_fock(fock_matrix_rhs_1pdm, ao_density_object, molecule, basis,
                                  eri_dict, dft_dict, pe_dict, timing_dict)
print("Fock matrix from 1PDM, from LinearSOlver")
print(fock_matrix_rhs_1pdm)
print()
print("Fock matrix from 1PDM, numpy.einsum")
print(2*F_1pdm_vlx)
#We have to take 0.5*fock_matrix_rhs_1pdm because there is a sqrt(2) in the excitation vectors
#(normalized? alpha+beta?, but we are looking only at the alpha-alpha block of the multipliers) 
rhs_1pdm_from_LinearSolver = numpy.einsum('pi,pa->ia', mo_occ,
                            numpy.einsum('ta,pt->pa', mo_vir, 0.5*fock_matrix_rhs_1pdm.alpha_to_numpy(0)))
print("\nIn MO basis, 1PDM contribution to the RHS, LinearSolver:")
print(rhs_1pdm_from_LinearSolver)
print()
print("Original, for comparison:")
print(vlx_1pdm_rhs)

Fock Type: Restricted 2J + K Matrix
Density Identifier: 0
[Dimension 7 x 7]
     0.00000000     0.00000000     0.00000000     0.00000000     0.00000000     0.00000000     0.00000000
     0.00000000     0.00000000     0.00000000     0.00000000     0.00000000     0.00000000     0.00000000
     0.00000000     0.00000000     0.00000000     0.00000000     0.00000000     0.00000000     0.00000000
     0.00000000     0.00000000     0.00000000     0.00000000     0.00000000     0.00000000     0.00000000
     0.00000000     0.00000000     0.00000000     0.00000000     0.00000000     0.00000000     0.00000000
     0.00000000     0.00000000     0.00000000     0.00000000     0.00000000     0.00000000     0.00000000
     0.00000000     0.00000000     0.00000000     0.00000000     0.00000000     0.00000000     0.00000000

Fock matrix from 1PDM, from LinearSOlver
Fock Type: Restricted general 2J + K Matrix
Density Identifier: 0
[Dimension 7 x 7]
    -0.64443158    -0.12478356    -0.02679243    -0.0267

In [46]:
ooov = adc1.reference_state.eri("o1o1o1v1").to_ndarray()
ovvv = adc1.reference_state.eri("o1v1v1v1").to_ndarray()
adcc_1pdm_rhs = ( -numpy.einsum('jk,ikja->ia', adcc_dm_oo, ooov, optimize=True) 
                 -numpy.einsum('bc,icba->ia', adcc_dm_vv, ovvv, optimize=True)
                )
print(adcc_1pdm_rhs)

[[-0.0187703 -0.0000359 -0.        -0.       ]
 [-0.08869    0.0000019 -0.        -0.       ]
 [ 0.000574   0.1057718 -0.        -0.       ]
 [-0.085727  -0.0001889 -0.        -0.       ]
 [-0.         0.        -0.        -0.       ]
 [-0.        -0.        -0.0187703 -0.0000359]
 [-0.        -0.        -0.08869    0.0000019]
 [-0.        -0.         0.000574   0.1057718]
 [-0.        -0.        -0.085727  -0.0001889]
 [-0.        -0.        -0.         0.       ]]


### 2. Contributions from the 2PDMs

\begin{equation}
F^\Gamma_{ia}=\sum_{\mu,\rho}C^\alpha_{\mu a}C^\alpha_{\rho i}\left[\sum_{\tau,\varphi}S_{\rho\tau}x^\alpha_{\tau\varphi}\sum_{\nu,\theta}x^\alpha_{\theta\nu}(\mu\nu|\theta\varphi)-\sum_{\tau,\nu}S_{\rho\tau}x^\alpha_{\tau\nu}\sum_{\theta,\varphi,\sigma} x^\sigma_{\theta\varphi}(\mu\nu|\theta\varphi)\right]-\sum_{\mu,\rho}C^\alpha_{\rho i}C^\alpha_{\mu a}\left[\sum_{\tau,\varphi}S_{\mu\tau}x^\alpha_{\tau\varphi}\sum_{\nu,\theta}x^\alpha_{\theta\nu}(\rho\nu|\theta\varphi)-\sum_{\tau,\nu} S_{\mu\tau}x^\alpha_{\tau\nu}\sum_{\theta,\varphi,\sigma}x^\sigma_{\theta\varphi}(\rho\nu|\theta\varphi) \right]
\end{equation}

Corrected Equation:

\begin{equation}
F^\Gamma_{ia}=\sum_{\mu,\zeta}C^\alpha_{\mu i}C^\alpha_{\zeta a}\sum_{\rho,\varphi}S_{\rho\zeta}x^\alpha_{\varphi\rho}\left[\sum_{\nu,\theta}x^\alpha_{\theta\nu}(\mu\nu|\theta\varphi)-\sum_{\theta,\nu,\sigma} x^\sigma_{\theta\nu}(\mu\varphi|\theta\nu)\right]-\sum_{\mu,\zeta}C^\alpha_{\mu i}C^\alpha_{\zeta a}\sum_{\rho,\varphi}S_{\mu\rho}x^\alpha_{\rho\varphi}\left[\sum_{\nu,\theta}x^\alpha_{\nu\theta}(\zeta\nu|\theta\varphi)-\sum_{\theta,\nu,\sigma}x^\sigma_{\nu\theta}(\zeta\varphi|\theta\nu) \right]
\end{equation}

In [47]:
# Care needs to be taken with the index order of the excitation vectors
FG1 = ( numpy.einsum('rz,mr->mz', ovlp,
                    numpy.einsum('pr,mp->mr', tda_eig_vec_ao,
                        (0.5*numpy.einsum('tn,mntp->mp',tda_eig_vec_ao,pqrs)
                        -numpy.einsum('tn,mptn->mp',tda_eig_vec_ao,pqrs))
                                )
                      )
            )
FG2 = ( numpy.einsum('mr,rz->mz', ovlp,
                    numpy.einsum('rp,zp->rz', tda_eig_vec_ao,
                        (0.5*numpy.einsum('nt,zntp->zp',tda_eig_vec_ao,pqrs)
                        -numpy.einsum('nt,zptn->zp',tda_eig_vec_ao,pqrs))
                                )
                      )
            )

#FG1_try2 =( numpy.einsum('rz,mr->mz', ovlp,
#                       numpy.einsum('pr,mp->mr', tda_eig_vec_ao,
#                                    numpy.einsum('tn,mntp->mp',tda_eig_vec_ao,pqrs)
#                                   )
#                      )
#            )
#FG2_try2 =( numpy.einsum('rz,mr->mz', ovlp,
#                       numpy.einsum('pr,mp->mr', tda_eig_vec_ao,
#                                    numpy.einsum('tn,mptn->mp',tda_eig_vec_ao,pqrs)
#                                   )
#                      )
#            )
#FG3_try2 =( numpy.einsum('mr,rz->mz', ovlp,
#                       numpy.einsum('rp,zp->rz', tda_eig_vec_ao,
#                                    numpy.einsum('nt,zntp->zp',tda_eig_vec_ao,pqrs)
#                                   )
#                      )
#            )
#FG4_try2 =( numpy.einsum('mr,rz->mz', ovlp,
#                       numpy.einsum('rp,zp->rz', tda_eig_vec_ao,
#                                    numpy.einsum('nt,zptn->zp',tda_eig_vec_ao,pqrs)
#                                   )
#                      )
#            )
#
#
#FG1_vlx =( numpy.einsum('rz,zm->rm', ovlp,
#                       numpy.einsum('zp,mp->zm', tda_eig_vec_ao,
#                                    numpy.einsum('tn,mntp->mp',tda_eig_vec_ao,pqrs)
#                                   )
#                      )
#            )
#FG1_vlx_ovlp =( numpy.einsum('rp,mp->rm',
#                numpy.einsum('rz,zp->rp', ovlp, tda_eig_vec_ao),
#                numpy.einsum('tn,mntp->mp',tda_eig_vec_ao,pqrs)
#                                   )
#            )
#
#FG2_vlx =  numpy.einsum('rz,zm->rm', ovlp,
#                       numpy.einsum('zn,nm->zm', tda_eig_vec_ao,
#                                    numpy.einsum('tp,mntp->nm',tda_eig_vec_ao,pqrs)
#                                   )
#                      )
#FG3_vlx =  numpy.einsum('mz,rz->rm', ovlp,
#                       numpy.einsum('zp,rp->rz', tda_eig_vec_ao,
#                                    numpy.einsum('tn,rntp->rp',tda_eig_vec_ao,pqrs)
#                                   )
#                      )
#FG4_vlx =  numpy.einsum('mz,rz->rm', ovlp,
#                       numpy.einsum('zn,rn->rz', tda_eig_vec_ao,
#                                    numpy.einsum('tp,rntp->rn',tda_eig_vec_ao,pqrs)
#                                   )
#                      )
print("eigenvector in MO:")
print(tda_eig_vec_as_mat)
print()
print("eigenvector in AO:")
print(tda_eig_vec_ao)
print()
# print("first term:")
# print(FG1_try2)
# print()
# print("first term:")
# print(FG1_vlx-FG2_vlx)
# print()
# F_2pdm_vlx_A = 0.5*FG1_try2 - FG2_try2 
# F_2pdm_vlx_B =-0.5*FG3_try2 + FG4_try2

# F_2pdm_vlx = F_2pdm_vlx_A + F_2pdm_vlx_B
#print(F_2pdm_vlx)
# another_vlx_2pdm_rhs = (numpy.einsum('mi,ma->ia', mo_occ, numpy.einsum('za,mz->ma', mo_vir, F_2pdm_vlx_A))
#                 +numpy.einsum('mi,ma->ia', mo_occ, numpy.einsum('za,mz->ma', mo_vir, F_2pdm_vlx_B))
#                )
# vlx_2pdm_rhs = (numpy.einsum('mi,ma->ia', mo_occ, numpy.einsum('za,mz->ma', mo_vir, F_2pdm_vlx)))
final_vlx_2pdm_rhs = (numpy.einsum('mi,ma->ia', mo_occ, numpy.einsum('za,mz->ma', mo_vir, FG1-FG2)))
# print(vlx_2pdm_rhs)
# print()
# print(another_vlx_2pdm_rhs)         
# print()
print(final_vlx_2pdm_rhs)

eigenvector in MO:
[[-0.        -0.       ]
 [ 0.         0.       ]
 [-0.         0.       ]
 [-0.         0.       ]
 [ 0.9999999  0.0004396]]

eigenvector in AO:
[[-0.         0.        -0.        -0.         0.         0.
   0.       ]
 [ 0.        -0.         0.         0.        -0.        -0.
  -0.       ]
 [-0.         0.        -0.        -0.         0.         0.
   0.       ]
 [-0.         0.        -0.        -0.         0.         0.
   0.       ]
 [ 0.133823  -0.8923462  0.8020473  0.8050707 -0.        -0.4181477
  -0.5948378]
 [-0.         0.        -0.        -0.         0.        -0.
   0.       ]
 [-0.         0.        -0.        -0.         0.         0.
   0.       ]]

[[ 0.0193264  0.0000085]
 [ 0.0251639  0.0000111]
 [-0.0001617 -0.0000001]
 [ 0.0567975  0.000025 ]
 [ 0.         0.       ]]


In [48]:
#2PDM contribution, using AODensityMatrix, AOFockMatrix, LinearSolver
#we need the transpose of the vector because of how the exchange part is calculated
#(mn|tp)!=(mt|np), but (mn|tp)=(nm|tp)=(mn|pt)=(nm|pt) real orbitals 
ao_density_2pdm_1 = AODensityMatrix([tda_eig_vec_ao.T], denmat.rest)
ao_density_2pdm_2 = AODensityMatrix([tda_eig_vec_ao], denmat.rest)
ao_fock_2pdm_1 = AOFockMatrix(ao_density_2pdm_1)
ao_fock_2pdm_2 =AOFockMatrix(ao_density_2pdm_2)
linear_solver_2pdm_1 = LinearSolver(comm, ostream) #Can we use the same object?
linear_solver_2pdm_1.comp_lr_fock(ao_fock_2pdm_1, ao_density_2pdm_1, molecule, basis,
                                  eri_dict, dft_dict, pe_dict, timing_dict)
linear_solver_2pdm_2 = LinearSolver(comm, ostream) #Can we use the same object?
linear_solver_2pdm_2.comp_lr_fock(ao_fock_2pdm_2, ao_density_2pdm_2, molecule, basis,
                                  eri_dict, dft_dict, pe_dict, timing_dict)
                        
fock_1 = (1.0*numpy.einsum('tn,mntp->mp',tda_eig_vec_ao,pqrs)
        -2.0*numpy.einsum('mn,mnpt->pt',tda_eig_vec_ao,pqrs))

fock_2 = (1.0*numpy.einsum('nt,zntp->zp',tda_eig_vec_ao,pqrs)
        -2.0*numpy.einsum('nt,zptn->zp',tda_eig_vec_ao,pqrs))

print("What we are after, 1st term:")
print(fock_1)
print()
print("What we get from comp_lr_fock using tda_eig_vec_ao.T:")
print(ao_fock_2pdm_1)
print()
print("What we are after, 2nd term:")
print(fock_2)
print()
print("What we get from comp_lr_fock using tda_eig_vec_ao:")
print(ao_fock_2pdm_2)
print()
LR_2pdm_F1 =  numpy.einsum('rz,mr->mz', ovlp,
             numpy.einsum('pr,mp->mr', tda_eig_vec_ao, 0.5*ao_fock_2pdm_1.alpha_to_numpy(0)))
LR_2pdm_F2 =  numpy.einsum('mr,rz->mz', ovlp,
                    numpy.einsum('rp,zp->rz', tda_eig_vec_ao, 0.5*ao_fock_2pdm_2.alpha_to_numpy(0)))

rhs_2pdm_from_LinearSolver = numpy.einsum('mi,ma->ia', mo_occ, 
                                          numpy.einsum('za,mz->ma', mo_vir, LR_2pdm_F2-LR_2pdm_F1))
print("2PDM RHS using LinearSolver:")
print(rhs_2pdm_from_LinearSolver)
print()
print("2PDM RHS using numpy.einsum:")
print(final_vlx_2pdm_rhs)

What we are after, 1st term:
[[ 0.         0.         0.         0.         0.0441064 -0.
  -0.       ]
 [ 0.         0.         0.         0.        -0.0444369  0.
   0.       ]
 [ 0.         0.         0.         0.         0.1539333  0.
  -0.       ]
 [ 0.         0.         0.         0.         0.1543664 -0.
   0.       ]
 [ 0.0142036  0.0625403  0.0208514  0.0208562  0.         0.0138765
   0.0197481]
 [ 0.        -0.         0.        -0.        -0.1718803  0.
   0.       ]
 [ 0.        -0.         0.         0.        -0.2446292  0.
   0.       ]]

What we get from comp_lr_fock using tda_eig_vec_ao.T:
Fock Type: Restricted general 2J + K Matrix
Density Identifier: 0
[Dimension 7 x 7]
    -0.00000000    -0.00000000     0.00000000     0.00000000    -0.04410642     0.00000000     0.00000000
    -0.00000000    -0.00000000    -0.00000000    -0.00000000     0.04443691    -0.00000000    -0.00000000
     0.00000000    -0.00000000    -0.00000000     0.00000000    -0.15393328    -0.00000

In [49]:
# Compare to adcc
adcc_2pdm_rhs = (numpy.einsum('jakb,ijkb->ia', adcc_DM_ovov, ooov, optimize=True)
                + numpy.einsum('ibjc,jcab->ia', adcc_DM_ovov, ovvv, optimize=True)
                )
print(adcc_2pdm_rhs)
#print(ooov)

[[ 0.0193264  0.0000283  0.         0.       ]
 [ 0.0251638  0.0000369  0.         0.       ]
 [-0.0002351 -0.0000003  0.         0.       ]
 [ 0.0567973  0.0000833  0.         0.       ]
 [ 0.         0.         0.         0.       ]
 [ 0.         0.         0.0193264  0.0000283]
 [ 0.         0.         0.0251638  0.0000369]
 [ 0.         0.        -0.0002351 -0.0000003]
 [ 0.         0.         0.0567973  0.0000833]
 [ 0.         0.         0.         0.       ]]


### INITIAL GUESS

In [50]:
# Neglect the non-diagonal part of the LHS,
# i.e. divide the RHS by the orbital-energy difference
vlx_rhs = final_vlx_2pdm_rhs + vlx_1pdm_rhs
lambda_guess = numpy.zeros((nocc,nvir))
mo_energies = scfdrv.scf_tensors['E']
#print(mo_energies[nocc:])
for i in range(nocc):
    ei=mo_energies[i]
    for a in range(nvir):
        ea=mo_energies[nocc+a]
        lambda_guess[i,a] = vlx_rhs[i,a]/(ei-ea)
print(lambda_guess)

[[-0.0000296  0.0000002]
 [ 0.0556239 -0.0000562]
 [-0.0004106 -0.1268601]
 [ 0.0641812  0.0001001]
 [-0.        -0.       ]]


In [51]:
# Compare to the analogous calculation with adcc
foo = adc1.reference_state.fock("o1o1").to_ndarray() 
fvv = adc1.reference_state.fock("v1v1").to_ndarray()

adcc_rhs = adcc_1pdm_rhs + adcc_2pdm_rhs
adcc_guess = adcc_rhs.copy() # rhs
for i in range(foo.shape[0]):
    for a in range(fvv.shape[0]):
        adcc_guess[i,a] = adcc_guess[i,a]/(foo[i,i]-fvv[a,a])

print(adcc_guess)

[[-0.0000296  0.0000004 -0.        -0.       ]
 [ 0.0556239 -0.0000305 -0.        -0.       ]
 [-0.0004828 -0.1268597 -0.        -0.       ]
 [ 0.0641815  0.0001813 -0.        -0.       ]
 [-0.        -0.        -0.        -0.       ]
 [-0.        -0.        -0.0000296  0.0000004]
 [-0.        -0.         0.0556239 -0.0000305]
 [-0.        -0.        -0.0004828 -0.1268597]
 [-0.        -0.         0.0641815  0.0001813]
 [-0.        -0.        -0.        -0.       ]]


In [52]:
#A L = RHS
# Transform the initial guess for lambda to the AO basis

lambda_guess_ao = numpy.matmul(mo_occ, numpy.matmul(lambda_guess, mo_vir.T))
print(lambda_guess_ao)

#print(numpy.matmul(mo_occ, numpy.matmul(adcc_guess[:nocc,:nvir], mo_vir.T))[:nocc+nvir,:nocc+nvir])

# Transform back to MO basis to check
lambda_guess_back_to_mo = numpy.matmul(mo_occ.T, numpy.matmul(ovlp, numpy.matmul(lambda_guess_ao, numpy.matmul(ovlp, mo_vir))))
print(lambda_guess_back_to_mo)

[[ 0.0006006 -0.0040049  0.0036161  0.0035968 -0.        -0.0018928
  -0.0026583]
 [-0.0005803  0.0038697 -0.0035375 -0.0034319  0.         0.0018714
   0.0025385]
 [-0.0036256  0.0241866 -0.0689463  0.0254071 -0.         0.0575336
  -0.0165559]
 [-0.0036461  0.0243019  0.0253902 -0.069179   0.        -0.0348374
   0.0488962]
 [-0.         0.        -0.        -0.         0.         0.
   0.       ]
 [-0.0046595  0.0310813 -0.0802085  0.0242542 -0.         0.0657215
  -0.0154662]
 [-0.0066138  0.0440934 -0.0026351 -0.0767936  0.        -0.0155451
   0.055003 ]]
[[-0.0000296  0.0000002]
 [ 0.0556239 -0.0000562]
 [-0.0004106 -0.1268601]
 [ 0.0641812  0.0001001]
 [-0.        -0.       ]]


### LHS: Contraction of the initial guess for $\boldsymbol{\lambda}$ with two-electron integrals

wrong equation, spin not integrated out, which needs to be done at the beginning!
\begin{equation}
    \sum_{\mu \nu} C_{\mu i} C_{\nu a}
    \sum_{\lambda \sigma} \big[
    \tilde{\lambda}_{\lambda \sigma} ( \mu \nu | \sigma \lambda )
    - \tilde{\lambda}_{\lambda \sigma} ( \mu \lambda | \sigma \nu )
        \big]
    %+ (\varepsilon_a - \varepsilon_i ) \gamma_{ia}
\end{equation}

using $p=j, q=b$ as well as $p=b, q=j$ with the condition $\lambda_{pq} = \lambda_{qp}$, the probably correct one is obtained:
\begin{equation}
    -\sum_{pq} \lambda_{pq} \big( [pq|ai] - [pi|aq] \big)
    = -\sum_{\varphi \theta} C_{\varphi i} C_{\theta a} \sum_{\mu \nu}
        \lambda_{\mu \nu} \big[ 4 (\mu \nu | \varphi \theta)
        - (\mu \theta | \varphi \nu) - (\nu \theta | \varphi \mu) \big]
\end{equation}

In [53]:
vlx_lhs1 = numpy.einsum('mn,mnpt->pt', lambda_guess_ao, pqrs)
vlx_lhs2 = numpy.einsum('mn,mtpn->pt', lambda_guess_ao, pqrs)
vlx_lhs3 = numpy.einsum('mn,ntpm->pt', lambda_guess_ao, pqrs)

vlx_lhs_ao = -4*vlx_lhs1 + vlx_lhs2 + vlx_lhs3

vlx_lhs_check = (-4*numpy.einsum('mn,mntp->pt', lambda_guess_ao, pqrs)
                    + 1.0*numpy.einsum('mn,mtpn->pt', lambda_guess_ao, pqrs)
                    + 1.0*numpy.einsum('mn,ntpm->pt', lambda_guess_ao, pqrs)
                      )
# Transform to MO
vlx_lhs_mo = numpy.matmul(mo_occ.T, numpy.matmul(vlx_lhs_ao, mo_vir))
#vlx_lhs_mo2 = numpy.einsum('pi,pa->ia', mo_occ, numpy.einsum('ta,pt->pa', mo_vir, vlx_lhs_ao))
print(vlx_lhs_mo)
print("Do we get the same result?")
print(vlx_lhs_ao)
print()
print(vlx_lhs_check)

[[-0.0056019  0.0000065]
 [-0.0157984  0.0000658]
 [-0.0000373  0.0028613]
 [-0.0079913  0.0000716]
 [-0.        -0.       ]]
Do we get the same result?
[[-0.2983001 -0.0601101 -0.0135209 -0.0135069 -0.        -0.0012683
  -0.0017896]
 [-0.0601101 -0.1241998 -0.0412683 -0.0411797  0.         0.0170832
   0.0242846]
 [-0.0135209 -0.0412683 -0.0542745 -0.0119132 -0.        -0.0215399
   0.0013912]
 [-0.0135069 -0.0411797 -0.0119132 -0.0541471 -0.         0.0084409
  -0.0197597]
 [-0.         0.        -0.        -0.        -0.147721  -0.
   0.       ]
 [-0.0012683  0.0170832 -0.0215399  0.0084409 -0.        -0.0748831
  -0.0204431]
 [-0.0017896  0.0242846  0.0013912 -0.0197597  0.        -0.0204431
  -0.0890619]]

[[-0.2983001 -0.0601101 -0.0135209 -0.0135069 -0.        -0.0012683
  -0.0017896]
 [-0.0601101 -0.1241998 -0.0412683 -0.0411797  0.         0.0170832
   0.0242846]
 [-0.0135209 -0.0412683 -0.0542745 -0.0119132 -0.        -0.0215399
   0.0013912]
 [-0.0135069 -0.0411797 -0.01191

In [54]:
#Lambda guess contracted with eri using AODensityMatrix, AOFockMatrix, LinearSolver
ao_density_lambda = AODensityMatrix([lambda_guess_ao], denmat.rest)
ao_fock_lambda = AOFockMatrix(ao_density_lambda)
linear_solver_lambda = LinearSolver(comm, ostream) #Can we use the same object?
linear_solver_lambda.comp_lr_fock(ao_fock_lambda, ao_density_lambda, molecule, basis,
                                  eri_dict, dft_dict, pe_dict, timing_dict)

L1_vlx = numpy.einsum('mn,mnpt->pt', lambda_guess_ao, pqrs)
L2_vlx = numpy.einsum('mn,mptn->pt', lambda_guess_ao, pqrs)
L_total_vlx = 2*L1_vlx - L2_vlx
print(ao_fock_lambda)
print()
print(L_total_vlx)
fock_lambda_numpy = ao_fock_lambda.alpha_to_numpy(0)
print()
print(fock_lambda_numpy)
lambda_mo_from_fock = ( numpy.matmul(mo_occ.T, numpy.matmul(fock_lambda_numpy, mo_vir)) 
                       + numpy.matmul(mo_vir.T, numpy.matmul(fock_lambda_numpy, mo_occ)).T )
print()
print("In MO, using the Fock Matrix object")
print(lambda_mo_from_fock)
print()
print("From the Equation:")
print(vlx_lhs_mo)

Fock Type: Restricted general 2J + K Matrix
Density Identifier: 0
[Dimension 7 x 7]
     0.14915003     0.03058762     0.00729448     0.00728824     0.00000000     0.00047832     0.00067119
     0.02952246     0.06209992     0.02885117     0.02882805     0.00000000    -0.00739723    -0.01049950
     0.00622639     0.01241716     0.02713727     0.00595862     0.00000000    -0.00847348     0.00022905
     0.00621868     0.01235166     0.00595454     0.02707356    -0.00000000     0.00308313    -0.00801932
     0.00000000    -0.00000000     0.00000000     0.00000000     0.07386052    -0.00000000    -0.00000000
     0.00078994    -0.00968595     0.03001341    -0.01152402     0.00000000     0.03744155     0.01020815
     0.00111846    -0.01378509    -0.00162025     0.02777905    -0.00000000     0.01023492     0.04453096


[[ 0.14915    0.0305876  0.0072945  0.0072882  0.         0.0004783
   0.0006712]
 [ 0.0295225  0.0620999  0.0288512  0.0288281  0.        -0.0073972
  -0.0104995]
 [ 0.006

In [55]:
#vlx_lhs3 = numpy.einsum('mn,mntp->pt', lambda_guess_ao, pqrs)
#vlx_lhs4 = numpy.einsum('mn,mptn->pt', lambda_guess_ao, pqrs)
#vlx_lhs_ao_try2 = -vlx_lhs3 + 0.5*vlx_lhs4
#vlx_1pdm_lhs = numpy.einsum('pi,pa->ia', mo_occ, numpy.einsum('ta,pt->pa', mo_vir, vlx_lhs_ao_try2))
#print(vlx_1pdm_lhs)

### Equation derived using $p=j, q=b$ and $p=b, q=j$:

\begin{equation}
-\sum_{pq}\lambda_{pq}\langle pa||qi\rangle = -\sum_{\theta\varphi}C^\alpha_{\theta a}C^\alpha_{\varphi i} \sum_{\mu\nu}(\mu\nu|\theta\varphi)\sum_{j,b,\sigma}C^\sigma_{\mu j} \lambda_{jb} C^\sigma_{\nu b}+\sum_{\theta\varphi}C^\alpha_{\theta a}C^\alpha_{\varphi i} \sum_{\mu\nu}(\mu\varphi|\theta\nu)\sum_{j,b}C^\alpha_{\mu j} \lambda_{jb} C^\alpha_{\nu b}\\
-\sum_{\theta\varphi}C^\alpha_{\theta a}C^\alpha_{\varphi i} \sum_{\mu\nu}(\nu\mu|\theta\varphi)\sum_{j,b,\sigma}C^\sigma_{\mu j} \lambda_{jb} C^\sigma_{\nu b}+\sum_{\theta\varphi}C^\alpha_{\theta a}C^\alpha_{\varphi i} \sum_{\mu\nu}(\nu\varphi|\theta\mu)\sum_{j,b}C^\alpha_{\mu j} \lambda_{jb} C^\alpha_{\nu b}\\
\\
=-\sum_{\theta\varphi}C^\alpha_{\theta a}C^\alpha_{\varphi i} \sum_{\mu\nu}(\mu\nu|\theta\varphi)\lambda^\sigma_{\mu\nu}+\sum_{\theta\varphi}C^\alpha_{\theta a}C^\alpha_{\varphi i} \sum_{\mu\nu}(\mu\varphi|\theta\nu)\lambda^\alpha_{\mu\nu}\\
-\sum_{\theta\varphi}C^\alpha_{\theta a}C^\alpha_{\varphi i} \sum_{\mu\nu}(\nu\mu|\theta\varphi)\lambda^\sigma_{\mu\nu}+\sum_{\theta\varphi}C^\alpha_{\theta a}C^\alpha_{\varphi i} \sum_{\mu\nu}(\nu\varphi|\theta\mu)\lambda^\alpha_{\mu\nu}\\
\\
=-2\sum_{\theta\varphi}C^\alpha_{\theta a}C^\alpha_{\varphi i} \sum_{\mu\nu,\sigma}(\mu\nu|\theta\varphi)\lambda^\sigma_{\mu\nu}+\sum_{\theta\varphi}C^\alpha_{\theta a}C^\alpha_{\varphi i} \sum_{\mu\nu}(\mu\varphi|\theta\nu)\lambda^\alpha_{\mu\nu}+\sum_{\theta\varphi}C^\alpha_{\theta a}C^\alpha_{\varphi i} \sum_{\mu\nu}(\nu\varphi|\theta\mu)\lambda^\alpha_{\mu\nu}
\end{equation}

In [56]:
vlx_lhs5 = numpy.einsum('mn,mnpt->pt', lambda_guess_ao, pqrs)
vlx_lhs6 = numpy.einsum('mn,mtpn->pt', lambda_guess_ao, pqrs)
vlx_lhs7 = numpy.einsum('mn,ntpm->pt', lambda_guess_ao, pqrs)
vlx_lhs_ao_try3 = -2*vlx_lhs5 + 0.5*vlx_lhs6 + 0.5*vlx_lhs7

vlx_1pdm_lhs_new = numpy.einsum('pi,pa->ia', mo_occ, numpy.einsum('ta,pt->pa', mo_vir, vlx_lhs_ao_try3))
print(vlx_1pdm_lhs_new)

[[-0.002801   0.0000032]
 [-0.0078992  0.0000329]
 [-0.0000187  0.0014306]
 [-0.0039956  0.0000358]
 [-0.        -0.       ]]


In [57]:
print("vlx_lhs_mo:")
print(vlx_lhs_mo)
print()
print("lambda_guess:")
print(lambda_guess)

vlx_lhs_mo:
[[-0.0056019  0.0000065]
 [-0.0157984  0.0000658]
 [-0.0000373  0.0028613]
 [-0.0079913  0.0000716]
 [-0.        -0.       ]]

lambda_guess:
[[-0.0000296  0.0000002]
 [ 0.0556239 -0.0000562]
 [-0.0004106 -0.1268601]
 [ 0.0641812  0.0001001]
 [-0.        -0.       ]]


In [58]:
# Add product of guess and eigenvalue differences to LHS
vlx_full_lhs = numpy.zeros((nocc,nvir))
for i in range(nocc):
    ei = mo_energies[i]
    for a in range(nvir):
        ea = mo_energies[nocc+a]
        vlx_full_lhs[i,a] = vlx_lhs_mo[i,a] + lambda_guess[i,a]*(ei-ea)

vlx_full_lhs_vec = vlx_full_lhs.reshape(nocc*nvir)
vlx_rhs_vec = vlx_rhs.reshape(nocc*nvir)

print(vlx_full_lhs_vec)
print()
print(vlx_rhs_vec)

[-0.0050459  0.0000017 -0.0793246  0.0001374  0.0002509  0.1086331
 -0.0369209  0.0000133 -0.         0.       ]

[ 0.0005561 -0.0000047 -0.0635262  0.0000716  0.0002882  0.1057718
 -0.0289296 -0.0000583  0.         0.       ]


In [59]:
from scipy.sparse import linalg

In [60]:
# Try conjugate gradient with identity matrix, full LHS and RHS
# identity = numpy.identity(nocc*nvir)

# solution, w = linalg.cg(A=identity, b=vlx_rhs_vec, x0=vlx_full_lhs_vec, tol=1e-6, maxiter=25)

# print(solution)
# print()
# print(w)

### Trying to define a linear Operator

In [61]:
# class vlx_lambda:
#     def __init__(self, guess, eri, scf_tensors, nocc):
#         """Parameters:
#         vlx_lambda: guess at current iteration for the lambda OV block
#         eri: two electron repulsion integrals in AO basis (chemists notation)
#         scf_tensors: from vlx scdrv
#         nocc: number of occupied orbitals
#         """
#         self.vector = guess
#         self.eri = eri
#         self.scf_tensors = scf_tensors
#         #print(scf_tensors)
#         self.nocc = nocc
        
def Ax(v):
    """Function to carry out matrix multiplication
    of Lagrange multipier vector with orbital Hessian
    matrix. Transforms guess to AO basis, carries out
    matrix-vector product and transforms back to MO.
    
    v: lambda at current iteration"""
    #print("Entering Ax(v)...:")
    #print(v)
    #print()
    #vlx_1pdm_lhs = 2.0*v
    #mo_energies = v.scf_tensors['E']
    #mo_occ = v.scf_tensors['C'][:,:nocc]
    #mo_vir = v.scf_tensors['C'][:,nocc:]
    #nvir = mo_vir.shape[1]
    current_lambda = v.reshape(nocc,nvir)
    print("VLX INPUT:")
    print(current_lambda)
    print()
    
    #vlx_lambda_2_in_ao = numpy.matmul(mo_occ, numpy.matmul(new_v, mo_vir.T))
    #L1_vlx = numpy.einsum('mn,mntp->pt', vlx_lambda_2_in_ao, pqrs)
    #L2_vlx = numpy.einsum('mn,mptn->pt', vlx_lambda_2_in_ao, pqrs)
    #L3_vlx = numpy.einsum('mn,nmtp->pt', vlx_lambda_2_in_ao, pqrs)
    #L4_vlx = numpy.einsum('mn,nptm->pt', vlx_lambda_2_in_ao, pqrs)
    #L_total_vlx = -2.0*L1_vlx + 1.0*L2_vlx - 2.0*L3_vlx + 1.0*L4_vlx
    #vlx_lambda_lhs_2 = numpy.einsum('pi,pa->ia', mo_occ, numpy.einsum('ta,pt->pa', mo_vir, L_total_vlx))
    #print("Vlx output using 4 terms:")
    #print(vlx_lambda_lhs_2)
    
    # Transform to AO
    vlx_lambda_ao = numpy.matmul(mo_occ, numpy.matmul(current_lambda, mo_vir.T))
 
    # Calculate the different contractions
    vlx_lhs_interm = (-4*numpy.einsum('mn,mntp->pt', vlx_lambda_ao, pqrs)
                    + 1.0*numpy.einsum('mn,mtpn->pt', vlx_lambda_ao, pqrs)
                    + 1.0*numpy.einsum('mn,ntpm->pt', vlx_lambda_ao, pqrs)
                      )
    # Transform back to MO
    vlx_1pdm_lhs = numpy.einsum('pi,pa->ia', mo_occ, numpy.einsum('ta,pt->pa', mo_vir, vlx_lhs_interm))
    print("OLD AX: what we get using numpy einsum:")
    print(vlx_1pdm_lhs)

# Add the diagonal part
    for i in range(nocc):
        ei = mo_energies[i]
        for a in range(nvir):
            ea = mo_energies[nocc+a]
            vlx_1pdm_lhs[i,a] += current_lambda[i,a]*(ei-ea) 
    print("VLX OUTPUT:")    
    print(vlx_1pdm_lhs)
    print()
    return vlx_1pdm_lhs.reshape(nocc*nvir)
    

In [62]:
def Ax_with_LinearSolver(v):
    """Function to carry out matrix multiplication
    of Lagrange multipier vector with orbital Hessian
    matrix, using AODensityMatrix, AOFockMatrix, and comp_lr_fock
    Transforms guess to AO basis, carries out
    matrix-vector product and transforms back to MO.
    
    v: lambda at current iteration"""
    
    current_lambda = v.reshape(nocc,nvir)
    print("LinearSolver VLX INPUT:")
    print(current_lambda)
    print()

    # Transform to AO
    vlx_lambda_ao = numpy.matmul(mo_occ, numpy.matmul(current_lambda, mo_vir.T))
    
    #Create AODensityMatrix object from lambda in AO
    vlx_ao_density_lambda = AODensityMatrix([vlx_lambda_ao], denmat.rest)
    
    #Create a Fock Matrix Object (initialized with zeros)
    vlx_fock_lambda = AOFockMatrix(vlx_ao_density_lambda)
    
    #Create linear solver object and compute the Fock matrix (contracts AODensityMatrix with ERI: 2J-K)
    linear_solver_lambda = LinearSolver(comm, ostream) #Can we use the same object?
    linear_solver_lambda.comp_lr_fock(vlx_fock_lambda, vlx_ao_density_lambda, molecule, basis,
                                      eri_dict, dft_dict, pe_dict, timing_dict)

    #Transform AOFockMatrix to numpy array (here we took only the alpha block)
    vlx_fock_lambda_numpy = vlx_fock_lambda.alpha_to_numpy(0)
    
    
    #Transform to MO basis (symmetrized w.r.t. occ. and virt.)
    vlx_lambda_mo_from_fock = -( numpy.matmul(mo_occ.T, numpy.matmul(vlx_fock_lambda_numpy, mo_vir)) 
                               + numpy.matmul(mo_vir.T, numpy.matmul(vlx_fock_lambda_numpy, mo_occ)).T )

#     print("What we get from LinearSolver:")
#     print(vlx_lambda_mo_from_fock)
#     #Calculate the different contractions
#     vlx_lhs_interm = (-4*numpy.einsum('mn,mntp->pt', vlx_lambda_ao, pqrs)
#                     + 1.0*numpy.einsum('mn,mtpn->pt', vlx_lambda_ao, pqrs)
#                     + 1.0*numpy.einsum('mn,ntpm->pt', vlx_lambda_ao, pqrs)
#                       )
#     # Transform back to MO
#     vlx_1pdm_lhs = numpy.einsum('pi,pa->ia', mo_occ, numpy.einsum('ta,pt->pa', mo_vir, vlx_lhs_interm))

#     print("What we get using numpy einsum:")
#     print(vlx_1pdm_lhs)
    
    # Add the diagonal part
    for i in range(nocc):
        ei = mo_energies[i]
        for a in range(nvir):
            ea = mo_energies[nocc+a]
            vlx_lambda_mo_from_fock[i,a] += current_lambda[i,a]*(ei-ea) 
    print("LinearSolver VLX OUTPUT:")    
    print(vlx_lambda_mo_from_fock)
    print()
    return vlx_lambda_mo_from_fock.reshape(nocc*nvir)

In [63]:
Ax_with_LinearSolver(lambda_guess.reshape(nocc*nvir))

LinearSolver VLX INPUT:
[[-0.0000296  0.0000002]
 [ 0.0556239 -0.0000562]
 [-0.0004106 -0.1268601]
 [ 0.0641812  0.0001001]
 [-0.        -0.       ]]

LinearSolver VLX OUTPUT:
[[-0.0050459  0.0000017]
 [-0.0793246  0.0001374]
 [ 0.0002509  0.1086331]
 [-0.0369209  0.0000133]
 [-0.         0.       ]]



array([-0.0050459,  0.0000017, -0.0793246,  0.0001374,  0.0002509,
        0.1086331, -0.0369209,  0.0000133, -0.       ,  0.       ])

In [64]:
#new_lambda_guess = vlx_lambda(lambda_guess, pqrs, scfdrv.scf_tensors, nocc)
# print("This is lambda guess")
vlx_guess_vec=lambda_guess.reshape(nocc*nvir)
# print(vlx_guess_vec)
# print()
# print("This is the rhs:")
# print(vlx_rhs_vec)
# print()
# print("This is Ax(lambda_guess)")
# print(Ax(v).shape)
# print()
A = linalg.LinearOperator((nocc*nvir,nocc*nvir), matvec=Ax)

print("Starting conjugate gradient:")
solution, w = linalg.cg(A=A, b=vlx_rhs_vec, x0=vlx_guess_vec, tol=1e-8, maxiter=25)

print("The solution in matrix form:")
print(solution.reshape(nocc,nvir))

VLX INPUT:
[[0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]]

OLD AX: what we get using numpy einsum:
[[0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]]
VLX OUTPUT:
[[0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]]

Starting conjugate gradient:
VLX INPUT:
[[-0.0000296  0.0000002]
 [ 0.0556239 -0.0000562]
 [-0.0004106 -0.1268601]
 [ 0.0641812  0.0001001]
 [-0.        -0.       ]]

OLD AX: what we get using numpy einsum:
[[-0.0056019  0.0000065]
 [-0.0157984  0.0000658]
 [-0.0000373  0.0028613]
 [-0.0079913  0.0000716]
 [-0.        -0.       ]]
VLX OUTPUT:
[[-0.0050459  0.0000017]
 [-0.0793246  0.0001374]
 [ 0.0002509  0.1086331]
 [-0.0369209  0.0000133]
 [-0.         0.       ]]

VLX INPUT:
[[-0.0000296  0.0000002]
 [ 0.0556239 -0.0000562]
 [-0.0004106 -0.1268601]
 [ 0.0641812  0.0001001]
 [-0.        -0.       ]]

OLD AX: what we get using numpy einsum:
[[-0.0056019  0.0000065]
 [-0.0157984  0.0000658]
 [-0.0000373  0.0028613]
 [-0.0079913  0.0000716]
 [-0.        -0.       ]]
VLX OUTPUT:
[[-0.0050

In [65]:
#Use Ax_with_LinearSolver for conjugate gradient
lr_vlx_guess_vec=lambda_guess.reshape(nocc*nvir)

print("Run Ax_with_LinearSolver once, lambda_guess as input:")
Ax_with_LinearSolver(lambda_guess.reshape(nocc*nvir))
print()

print("Create scipy.linalg LinearOperator:")
LR_A = linalg.LinearOperator((nocc*nvir,nocc*nvir), matvec=Ax_with_LinearSolver)

LR_full_RHS_1d_vec = (rhs_2pdm_from_LinearSolver + rhs_1pdm_from_LinearSolver).reshape(nocc*nvir)

print("Starting conjugate gradient...")
LR_solution, lr_w = linalg.cg(A=LR_A, b=LR_full_RHS_1d_vec, x0=LR_full_RHS_1d_vec.copy(), tol=1e-8, maxiter=25)

print("The solution in matrix form:")
print(LR_solution.reshape(nocc,nvir))
print()
print("The solution using numpy.einsum:")
print(solution.reshape(nocc,nvir))
#z1 = [[ 0.0007725 -0.000002 ]
# [ 0.120848  -0.0000164]
# [-0.0010734 -0.3172186]
# [ 0.0731763  0.0005631]
# [-0.         0.       ]]

Run Ax_with_LinearSolver once, lambda_guess as input:
LinearSolver VLX INPUT:
[[-0.0000296  0.0000002]
 [ 0.0556239 -0.0000562]
 [-0.0004106 -0.1268601]
 [ 0.0641812  0.0001001]
 [-0.        -0.       ]]

LinearSolver VLX OUTPUT:
[[-0.0050459  0.0000017]
 [-0.0793246  0.0001374]
 [ 0.0002509  0.1086331]
 [-0.0369209  0.0000133]
 [-0.         0.       ]]


Create scipy.linalg LinearOperator:
LinearSolver VLX INPUT:
[[0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]]

LinearSolver VLX OUTPUT:
[[-0. -0.]
 [-0. -0.]
 [-0. -0.]
 [-0. -0.]
 [-0. -0.]]

Starting conjugate gradient...
LinearSolver VLX INPUT:
[[ 0.0005561 -0.0000047]
 [-0.0635262  0.0000716]
 [ 0.0002882  0.1057718]
 [-0.0289296 -0.0000583]
 [-0.        -0.       ]]

LinearSolver VLX OUTPUT:
[[-0.006405   0.0000792]
 [ 0.0779006 -0.0001299]
 [-0.0001823 -0.0889584]
 [ 0.030791  -0.0000223]
 [-0.        -0.       ]]

LinearSolver VLX INPUT:
[[ 0.0005561 -0.0000047]
 [-0.0635262  0.0000716]
 [ 0.0002882  0.1057718]
 [-0.0289296 -0.0000

In [66]:
AOVOV = numpy.ones(ovov.shape)   
adcc_nocc = foo.shape[0]
adcc_nvirt = fvv.shape[0]
oovv = adc1.reference_state.eri("o1o1v1v1").to_ndarray()
#adcc_A = np.zeros((adcc_nocc*adcc_nvirt, adcc_nocc*adcc_nvirt))
#these are supposed to be the elements that multiply lambda_jb
AOVOV = ( numpy.einsum('iajb,ibja->iajb', AOVOV, ovov, optimize=True)
         -numpy.einsum('iajb,ijab->iajb', AOVOV, oovv, optimize=True) 
        )
#print("OOVV\n")
#print(oovv)
#print()
for i in range(adcc_nocc):
    for a in range(adcc_nvirt):
        AOVOV[i,a,i,a] += foo[i,i]-fvv[a,a] #check signs!

adcc_A = AOVOV.reshape(adcc_nocc*adcc_nvirt,adcc_nocc*adcc_nvirt)

In [67]:
adcc_1d_guess = numpy.reshape(adcc_guess, (adcc_nocc*adcc_nvirt))
adcc_1d_rhs = numpy.reshape(adcc_rhs, (adcc_nocc*adcc_nvirt))

# print("MATRIX A:\n")
# print(adcc_A)
# print()
# print("GUESS:\n")
# print(adcc_1d_guess)
# print()
# print("RHS:\n")
# print(adcc_1d_rhs)
# print()

lov, w = linalg.cg(A=adcc_A, b=adcc_1d_rhs, x0=adcc_1d_guess, tol=1e-8, maxiter=25)

In [68]:
def adcc_Ax_MO(v):
    print("Entering Ax_MO(v)...:")
    #print(v)
    #print()
    new_v = v.reshape(adcc_nocc,adcc_nvirt)
    print("ADCC INPUT")
    print(new_v)
    print()
    adcc_1pdm_lhs = ( numpy.einsum('kb,kaib->ia', new_v, ovov, optimize=True)
         -numpy.einsum('kb,kiba->ia', new_v, oovv, optimize=True) 
        )

    # Add the diagonal part
    for i in range(adcc_nocc):
        ei = foo[i,i]
        for a in range(adcc_nvirt):
            ea = fvv[a,a]
            adcc_1pdm_lhs[i,a] += new_v[i,a]*(ei-ea)
    print("ADCC OUTPUT")
    print(adcc_1pdm_lhs)
    print()
    return adcc_1pdm_lhs.reshape(adcc_nocc*adcc_nvirt)

adcc_Ax = linalg.LinearOperator((adcc_nocc*adcc_nvirt,adcc_nocc*adcc_nvirt), matvec=adcc_Ax_MO)
adcc_solution, adcc_w = linalg.cg(A=adcc_Ax, b=adcc_1d_rhs, x0=adcc_1d_guess, tol=1e-8, maxiter=25)

Entering Ax_MO(v)...:
ADCC INPUT
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]

ADCC OUTPUT
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]

Entering Ax_MO(v)...:
ADCC INPUT
[[-0.0000296  0.0000004 -0.        -0.       ]
 [ 0.0556239 -0.0000305 -0.        -0.       ]
 [-0.0004828 -0.1268597 -0.        -0.       ]
 [ 0.0641815  0.0001813 -0.        -0.       ]
 [-0.        -0.        -0.        -0.       ]
 [-0.        -0.        -0.0000296  0.0000004]
 [-0.        -0.         0.0556239 -0.0000305]
 [-0.        -0.        -0.0004828 -0.1268597]
 [-0.        -0.         0.0641815  0.0001813]
 [-0.        -0.        -0.        -0.       ]]

ADCC OUTPUT
[[-0.0050459 -0.0000027  0.         0.       ]
 [-0.0793245  0.0001212  0.         0.       ]
 [ 0.0002806  0.1086329  0.         0.  

In [69]:
print("MO basis, ADCC solution, regular CG")
print(lov.reshape(adcc_nocc,adcc_nvirt))
print()
print("MO basis, ADCC solution, Linear Operator")
print(adcc_solution.reshape(adcc_nocc, adcc_nvirt))
print()
print("AO basis, VLX solution, linear operator")
print(solution.reshape(nocc,nvir))

MO basis, ADCC solution, regular CG
[[-0.0001184  0.0000004 -0.        -0.       ]
 [ 0.0609715  0.0000338 -0.        -0.       ]
 [-0.0008686 -0.2091809 -0.        -0.       ]
 [-0.2370318  0.0007657 -0.        -0.       ]
 [-0.        -0.        -0.        -0.       ]
 [-0.        -0.        -0.0001184  0.0000004]
 [-0.        -0.         0.0609715  0.0000338]
 [-0.        -0.        -0.0008686 -0.2091809]
 [-0.        -0.        -0.2370318  0.0007657]
 [-0.        -0.        -0.        -0.       ]]

MO basis, ADCC solution, Linear Operator
[[-0.0001184  0.0000004 -0.        -0.       ]
 [ 0.0609715  0.0000338 -0.        -0.       ]
 [-0.0008686 -0.2091809 -0.        -0.       ]
 [-0.2370318  0.0007657 -0.        -0.       ]
 [-0.        -0.        -0.        -0.       ]
 [-0.        -0.        -0.0001184  0.0000004]
 [-0.        -0.         0.0609715  0.0000338]
 [-0.        -0.        -0.0008686 -0.2091809]
 [-0.        -0.        -0.2370318  0.0007657]
 [-0.        -0.        -0. 

In [70]:
#solution, w = linalg.cg(A=A, b=vlx_rhs_vec, x0=vlx_guess_vec, tol=1e-8, maxiter=25)
#adcc_solution, adcc_w = linalg.cg(A=adcc_Ax, b=adcc_1d_rhs, x0=adcc_1d_guess, tol=1e-8, maxiter=25)
print("VLX RHS:\n")
print(vlx_rhs_vec)
print()
print("ADCC RHS:\n")
print(adcc_1d_rhs)
print()
print("VLX GUESS:\n")
print(vlx_guess_vec)
print()
print("ADCC GUESS:\n")
print(adcc_1d_guess)
print()
vlx_apply_Ax = Ax(vlx_guess_vec)
adcc_apply_Ax = adcc_Ax_MO(adcc_1d_guess)
print("VLX Ax(v)\n")
print(vlx_apply_Ax)
print()
print("ADCC Ax(v)\n")
print(adcc_apply_Ax)
print()

VLX RHS:

[ 0.0005561 -0.0000047 -0.0635262  0.0000716  0.0002882  0.1057718
 -0.0289296 -0.0000583  0.         0.       ]

ADCC RHS:

[ 0.000556  -0.0000075  0.         0.        -0.0635262  0.0000388
  0.         0.         0.0003389  0.1057715  0.         0.
 -0.0289297 -0.0001056  0.         0.         0.         0.
  0.         0.         0.         0.         0.000556  -0.0000075
  0.         0.        -0.0635262  0.0000388  0.         0.
  0.0003389  0.1057715  0.         0.        -0.0289297 -0.0001056
  0.         0.         0.         0.       ]

VLX GUESS:

[-0.0000296  0.0000002  0.0556239 -0.0000562 -0.0004106 -0.1268601
  0.0641812  0.0001001 -0.        -0.       ]

ADCC GUESS:

[-0.0000296  0.0000004 -0.        -0.         0.0556239 -0.0000305
 -0.        -0.        -0.0004828 -0.1268597 -0.        -0.
  0.0641815  0.0001813 -0.        -0.        -0.        -0.
 -0.        -0.        -0.        -0.        -0.0000296  0.0000004
 -0.        -0.         0.0556239 -0.0000305

In [71]:
# Calculate the total density (including SCF part)
# and calculate the relaxed excited-state dipole moment

# dipole integrals
dipole_drv = vlx.ElectricDipoleIntegralsDriver(comm)
dipole_mats = dipole_drv.compute(molecule, basis)
dipole_ints = (dipole_mats.x_to_numpy(),
              dipole_mats.y_to_numpy(),
              dipole_mats.z_to_numpy())


#vlx_lambda_ao_conv = numpy.matmul(mo_occ, numpy.matmul(solution.reshape(nocc,nvir), mo_vir.T))
vlx_lambda_ao_conv = numpy.matmul(mo_occ, numpy.matmul(LR_solution.reshape(nocc,nvir), mo_vir.T))

# electronic contribution
# factor 4 because 2 for ov and vo blocks and 2 for alpha & beta spin
total_density = 2*scfdrv.scf_tensors['D'][0] + vlx_dm_ao + 4*vlx_lambda_ao_conv
adcc_total_density = 2*scfdrv.scf_tensors['D'][0] + adcc_dm_ao 
electronic_dipole = -1.0 * numpy.array(
    [numpy.sum(dipole_ints[d] * total_density) for d in range(3)])
adcc_electronic_dipole = -1.0 * numpy.array(
    [numpy.sum(dipole_ints[d] * adcc_total_density) for d in range(3)])

# nuclear contribution
coords = molecule.get_coordinates()
nuclear_charges = molecule.elem_ids_to_numpy()
nuclear_dipole = numpy.sum(coords.T * nuclear_charges, axis=1)

dipole_moment = (nuclear_dipole + electronic_dipole)
adcc_dipole_moment = (nuclear_dipole + adcc_electronic_dipole)

print("dipole moment [au] =", dipole_moment)
print()
#print("adcc dm dipole moment [au] =", adcc_dipole_moment)
#print()
print("CIS/STO-3G relaxed dipole moment for water in Q-Chem:")
print("Dip. moment [a.u.]: [-0.0184062, 0.00000, -0.0076203]")
print("Total dipole: 0.020 au, 0.051 Debye")
print()
print("CIS/6-31G relaxed dipole moment for water in Q-Chem:")
print("Dip. moment [a.u.]:  [-0.0424424,  0.00000,   -0.0256919]")
print("Total dipole: 0.050 au, 0.126 Debye")

dipole moment [au] = [-0.122634   0.        -0.0835126]

CIS/STO-3G relaxed dipole moment for water in Q-Chem:
Dip. moment [a.u.]: [-0.0184062, 0.00000, -0.0076203]
Total dipole: 0.020 au, 0.051 Debye

CIS/6-31G relaxed dipole moment for water in Q-Chem:
Dip. moment [a.u.]:  [-0.0424424,  0.00000,   -0.0256919]
Total dipole: 0.050 au, 0.126 Debye
